# 4 Lstm Model

In [2]:
# functions


from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from pre_processing import load_preprocessed_data
from evaluator import evaluate_model



def build_lstm_model(input_shape):
    """
    Builds and compiles an LSTM model for RUL prediction.

    Parameters:
        input_shape (tuple): Shape of input data (timesteps, features)

    Returns:
        model (keras.Model): Compiled LSTM model
    """
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(32))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))

    model.compile(optimizer='adam', loss='mse')

    return model

def train_lstm_model(model, X_train, y_train, X_val, y_val, epochs=20, batch_size=64):
    """
    Trains the LSTM model with training and validation data.

    Parameters:
        model (keras.Model): Compiled LSTM model
        X_train, y_train: Training data
        X_val, y_val: Validation data
        epochs (int): Number of epochs
        batch_size (int): Batch size

    Returns:
        model: Trained Keras model
        history: Training history object
    """
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stop],
        verbose=1
    )

    return model, history

def predict_lstm_model(model, X):
    """
    Generates RUL predictions from the trained LSTM model.

    Parameters:
        model (keras.Model): Trained LSTM model
        X (np.ndarray): Input data to predict on

    Returns:
        np.ndarray: Predicted RUL values
    """
    return model.predict(X).flatten()

def run_lstm_pipeline(npz_path="fd001_last.npz", epochs=20, batch_size=64):
    """
    Full pipeline to train, evaluate, and report LSTM model performance.

    Parameters:
        npz_path (str): Path to .npz file with preprocessed data
        epochs (int): Number of training epochs
        batch_size (int): Size of each training batch

    Returns:
        Tuple: model, y_val, y_pred, evaluation_results
    """
    print("=== Step 1: Load Preprocessed Data ===")
    X_train, y_train, X_val, y_val, X_test, y_test = load_preprocessed_data(npz_path)
    print(f"Train shape: {X_train.shape}, {y_train.shape}")
    print(f"Val shape  : {X_val.shape}, {y_val.shape}")
    print("")

    print("=== Step 2: Build Model ===")
    input_shape = X_train.shape[1:]
    model = build_lstm_model(input_shape)
    model.summary()
    print("")

    print("=== Step 3: Train Model ===")
    model, history = train_lstm_model(model, X_train, y_train, X_val, y_val,
                                      epochs=epochs, batch_size=batch_size)
    print("Training complete.\n")

    print("=== Step 4: Predict ===")
    y_pred = predict_lstm_model(model, X_val)
    print("Sample predictions:", y_pred[:5])
    print("")

    print("=== Step 5: Evaluate ===")
    evaluation_results = evaluate_model(y_val, y_pred, model_name="LSTM")
    print("")

    return model, y_val, y_pred, evaluation_results

def save_lstm_model(model, filename="lstm_model.h5"):
    """
    Saves the trained LSTM model to an HDF5 (.h5) file.

    Parameters:
        model (keras.Model): Trained model to save
        filename (str): Filename for saving
    """
    model.save(filename)
    print(f"Model saved to {filename}")


def load_lstm_model(filename="lstm_model.h5"):
    """
    Loads a saved LSTM model from an HDF5 (.h5) file.

    Parameters:
        filename (str): Filename of the saved model

    Returns:
        keras.Model: Loaded model
    """
    from keras.models import load_model
    model = load_model(filename)
    print(f"Model loaded from {filename}")
    return model

In [6]:
# build_lstm_model testing 

X_train, y_train, X_val, y_val, X_test, y_test = load_preprocessed_data("fd001_last.npz")

# infer shape from training data
input_shape = X_train.shape[1:]  # (timesteps, features)

# build model
model = build_lstm_model(input_shape)

# summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 30, 64)            20480     
                                                                 
 dropout_2 (Dropout)         (None, 30, 64)            0         
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 32,929
Trainable params: 32,929
Non-trainable params: 0
_________________________________________________________________


In [7]:
# train_lstm_model testing 
model, history = train_lstm_model(model, X_train, y_train, X_val, y_val)

Epoch 1/20
222/222 [==============================] - 7s 22ms/step - loss: 7097.2627 - val_loss: 6296.0713
Epoch 2/20
222/222 [==============================] - 4s 20ms/step - loss: 5936.3330 - val_loss: 5380.7710
Epoch 3/20
222/222 [==============================] - 4s 20ms/step - loss: 5089.8066 - val_loss: 4608.9536
Epoch 4/20
222/222 [==============================] - 5s 20ms/step - loss: 4365.3662 - val_loss: 3947.1873
Epoch 5/20
222/222 [==============================] - 4s 20ms/step - loss: 3748.3806 - val_loss: 3376.8721
Epoch 6/20
222/222 [==============================] - 4s 20ms/step - loss: 3202.3462 - val_loss: 2881.4116
Epoch 7/20
222/222 [==============================] - 4s 20ms/step - loss: 2746.2454 - val_loss: 2457.7581
Epoch 8/20
222/222 [==============================] - 4s 20ms/step - loss: 2338.4438 - val_loss: 2092.7510
Epoch 9/20
222/222 [==============================] - 5s 20ms/step - loss: 1994.4727 - val_loss: 1772.4889
Epoch 10/20
222/222 [================

In [9]:
# predict_lstm_model testing

y_pred = predict_lstm_model(model, X_val)
print(y_pred[:5])


111/111 [==============================] - 1s 5ms/step
[ 77.676186  87.23597  104.237206 104.23795  102.93428 ]


In [10]:
# see how good my model is 


from evaluator import evaluate_model

results = evaluate_model(y_val, y_pred, model_name="LSTM")


LSTM Evaluation:
  RMSE: 19.0343
  MAE : 15.0567


In [12]:
model, y_val, y_pred, results = run_lstm_pipeline("fd001_last.npz")

=== Step 1: Load Preprocessed Data ===
Train shape: (14184, 30, 15), (14184,)
Val shape  : (3547, 30, 15), (3547,)

=== Step 2: Build Model ===
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 30, 64)            20480     
                                                                 
 dropout_4 (Dropout)         (None, 30, 64)            0         
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 32,929
Trainable params: 32,

In [14]:
save_lstm_model(model, "my_lstm_fd001.h5")

Model saved to my_lstm_fd001.h5


In [15]:
model = load_lstm_model("my_lstm_fd001.h5")

Model loaded from my_lstm_fd001.h5


In [ ]:
def prepare_lstm_data(file_path, seq_len=30, save_as="fd001_last.npz"):
    """
    Prepares LSTM-ready 3D data from raw C-MAPSS file using pre_processing module.

    Parameters:
        file_path (str): Path to raw FD001 text file.
        seq_len (int): Length of each time window.
        save_as (str): Filename to save the .npz file.

    Returns:
        Tuple: X_train, y_train, X_val, y_val
    """
    # Step 1: Load raw data
    df = data_loader.load_raw_data(file_path)
    
    # Step 2: Drop flat sensors
    df = pre_processing.drop_flat_sensors(df)
    
    # Step 3: Calculate RUL
    df = pre_processing.calculate_rul(df)
    
    # Step 4: Standardise
    df = pre_processing.standardise_per_condition(df)
    
    # Step 5: Generate sliding windows
    X, y = pre_processing.generate_sliding_windows(df, seq_len=seq_len)
    
    # Step 6: Train/val split
    from sklearn.model_selection import train_test_split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Step 7: Save
    pre_processing.save_preprocessed_data(X_train, y_train, X_val, y_val, X_test=None, y_test=None, filename=save_as)
    
    return X_train, y_train, X_val, y_val